## Набор данных Футболисты

В качестве задания вам предлагается повторить данные действия для набора данных
- [Blood Transfusion Service Center Data Set](http://archive.ics.uci.edu/ml/datasets/Blood+Transfusion+Service+Center)

In [ ]:
import pandas as pd

Набор данных о футболистах.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/yakushinav/omo/main/data/data_football_profile.csv', sep='\t')

In [ ]:
df.columns

Функция возвращает DataFrame (то есть таблицу), однако затем приобретает ещё много важных параметров, среди которых:

* sep — разделитель данных, по умолчанию ',';
* decimal — разделитель числа на целую и дробную часть, по умолчанию'.';
* names — список с названиями колонок, не обязательный параметр;
* skiprows — если файл содержит системную информацию, можно просто её пропустить. Необязательный параметр.

С помощью функции ```head``` можем посмотреть на первые несколько строк нашего датасета:

In [ ]:
df.head()

Удалим колонку Value, к которой мы не знаем точную интерпретацию:

In [ ]:
df.drop(['Value'], axis=1, inplace=True)

In [ ]:
# последние несколько строк:
df.tail(3)

Посмотрим на размер нашего датасета. Первое число – количество строк (наблюдений), второе – количество столбцов (признаков):

In [ ]:
df.shape

Если вы хотите переименовать какую-то переменную, воспользуйтесь ```rename```:

In [ ]:
df.rename({'Wage' : 'Salary'}, axis='columns', inplace=True)

In [ ]:
df.columns

Давайте посмотрим на информацию о датасете. В .info() можно передать дополнительные параметры, среди которых:

* verbose: печатать ли информацию о DataFrame полностью (если таблица очень большая, то некоторая информация может потеряться);
* memory_usage: печатать ли потребление памяти (по умолчанию используется True, но можно поставить либо False, что уберёт потребление памяти, либо 'deep' , что подсчитает потребление памяти более точно);
* null_counts: подсчитывать ли количество пустых элементов (по умолчанию True).

In [ ]:
df.info()

Можно вывести только тип данных в каждой колонке:

In [ ]:
df.dtypes

NameError: name 'df' is not defined

Заметим, что зарплата у нас записана строкой.

In [ ]:
def fix_wage(x):
    # отрезаем $
    x = x[:-1]
    # меняем запятую на точку
    x = x.replace(',', '.')
    return float(x)

In [ ]:
df['Salary'] = df['Salary'].apply(fix_wage)

Метод describe показывает основные статистические характеристики данных по каждому числовому признаку (типы int64 и float64): число непропущенных значений, среднее, стандартное отклонение, диапазон, медиану, 0.25 и 0.75 квартили.

In [ ]:
df.describe()

Чтобы посмотреть статистику по нечисловым признакам (например, по строчным (object) или булевым (bool) данным), нужно явно указать интересующие нас типы в параметре метода describe include:

In [ ]:
df.describe(include = ['object'])

Было бы полезно узнать, много ли у нас пропусков в датасете.

In [ ]:
df.isna().sum()
print(df.dtypes)

Чтобы удалить пропуски из данных, нужно вопспользоваться ```df.dropna()```, либо заполнить их значениями (например, средним) -  ```df.fillna(df['column_name'].mean())``` .
Если в датасете содержатся дубликаты строк - воспользуйтесь методом ```df.drop_duplicates()```.

In [ ]:
# заполним количественные переменные средними значениями (медианой)
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Salary'] = df['Salary'].fillna(df['Salary'].median())

# у оставшихся переменных удалим строки с пропусками
df.dropna(inplace=True)

df.isna().sum()

Выведем уникальные значения по возрасту и сколько раз каждое из них встречается в датасете (по убыванию).

In [ ]:
df['Age'].value_counts()

Чтобы вывести уникальные значения в столбце или их количество, нужно использовать ```unique``` и ```nunique``` соответственно. Посмотрим, сколько у нас уникальных футбольных клубов.

In [ ]:
print('Всего {} футбольных клубов'.format(df['Club'].nunique()))

In [ ]:
df['Club'].unique()[:10]

Посмотрим на среднюю зарплату по клубу:

In [ ]:
grouped = df.groupby('Club', as_index=False)['Salary'].mean()
# добавим сортировку по убыванию
grouped.sort_values(by='Salary', ascending=False)

Добавим еще подсчет минимума, максимума и медианы по каждой группе:

In [ ]:
df.groupby('Club')['Salary'].agg(['mean', 'min', 'max', 'median'])

Сгруппируем одновременно по стране и клубу:

In [ ]:
df.groupby(['Nationality', 'Club'], as_index=False)['Salary'].mean()

Добавим сортировку внутри групп:

In [ ]:
df.groupby(['Nationality', 'Club']).apply(lambda x: x.sort_values(by='Salary', ascending=False))

Теперь удалим лишние колонки. Обратите внимание на обратный слэш, это line continuation character.

In [ ]:
df.groupby(['Nationality', 'Club']).apply(lambda x: x.sort_values(by='Salary', ascending=False)).\
                                                    drop(['Nationality', 'Club'], axis=1)

Посчитаем арифметическое среднее, моду и медиану возраста футболистов (количественной переменной):

In [ ]:
print('Среднее:', round(df['Age'].mean(), 2),
      'Медиана:', df['Age'].median(),
      'Мода:', df['Age'].mode()[0])

Для качественных переменных с помощью pandas можно вывести моду. Посмотрим на самую часто встречающуюся национальность:

In [ ]:
df['Nationality'].mode()

Часто возникает необходимость выбрать данные из DataFrame по определённому условию. Например, если в уже известном нам наборе данных о футболистах мы хотим выбрать только тех, у кого возраст больше 20 лет, используется следующий код:

In [ ]:
df[df.Age > 20]

Выберем футболистов, возраст которых больше среднего возраста футболистов, при условии, что они принадлежат ФК Барселона (Club == 'FC Barcelona').

In [ ]:
df[(df.Age > df.Age.mean()) & (df.Club == 'FC Barcelona')]

Чтобы объединить данные из нескольких датасетов по ключу (общей колонке), в pandas можно воспользовать встроенными аналогами SQL методов. В метод ```join``` в качестве аргумента how нужно указать тип объединения датасетов: inner, outer, left или right.

In [ ]:
df_info = pd.read_csv('https://raw.githubusercontent.com/yakushinav/omo/main/data/data_football_info.csv', sep='\t')

joined_dfs = df_info.set_index('Name').join(df.set_index('Name'), how='inner').reset_index()
joined_dfs.head(5)

В качестве задания вам предлагается повторить данные действия по визуализации и классификации для набора данных
- [Blood Transfusion Service Center Data Set](http://archive.ics.uci.edu/ml/datasets/Blood+Transfusion+Service+Center)

In [1]:
pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo
blood_transfusion_service_center = fetch_ucirepo(id=176)
X = blood_transfusion_service_center.data.features
y = blood_transfusion_service_center.data.targets
print(blood_transfusion_service_center.metadata)
print(blood_transfusion_service_center.variables)


{'uci_id': 176, 'name': 'Blood Transfusion Service Center', 'repository_url': 'https://archive.ics.uci.edu/dataset/176/blood+transfusion+service+center', 'data_url': 'https://archive.ics.uci.edu/static/public/176/data.csv', 'abstract': 'Data taken from the Blood Transfusion Service Center in Hsin-Chu City in Taiwan -- this is a classification problem. ', 'area': 'Business', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 748, 'num_features': 4, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Donated_Blood'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2008, 'last_updated': 'Sat Mar 16 2024', 'dataset_doi': '10.24432/C5GS39', 'creators': ['I-Cheng Yeh'], 'intro_paper': {'ID': 434, 'type': 'NATIVE', 'title': 'Knowledge discovery on RFM model using Bernoulli sequence', 'authors': 'I. Yeh, K. Yang, Tao-Ming Ting', 'venue': 'Expert systems with applications', 'year': 2009, 'journa

In [7]:
import pandas as pd
data_url = 'https://archive.ics.uci.edu/static/public/176/data.csv'
try:
    df = pd.read_csv(data_url)
    print(df.head())
except Exception as e:
    print(f"Ошибка: {e}")

   Recency  Frequency  Monetary  Time  Donated_Blood
0        2         50     12500    98              1
1        0         13      3250    28              1
2        1         16      4000    35              1
3        2         20      5000    45              1
4        1         24      6000    77              0


In [8]:
df.columns

Index(['Recency', 'Frequency', 'Monetary', 'Time', 'Donated_Blood'], dtype='object')

In [9]:
df.head()

,Recency,Frequency,Monetary,Time,Donated_Blood
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [10]:
df.tail(3)

,Recency,Frequency,Monetary,Time,Donated_Blood
745,23,3,750,62,0
746,39,1,250,39,0
747,72,1,250,72,0


In [11]:
df.shape

(748, 5)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Recency        748 non-null    int64
 1   Frequency      748 non-null    int64
 2   Monetary       748 non-null    int64
 3   Time           748 non-null    int64
 4   Donated_Blood  748 non-null    int64
dtypes: int64(5)
memory usage: 29.3 KB


In [13]:
df.dtypes

,0
Recency,int64
Frequency,int64
Monetary,int64
Time,int64
Donated_Blood,int64


In [14]:
df.describe()

,Recency,Frequency,Monetary,Time,Donated_Blood
count,748.000000,748.000000,748.000000,748.000000,748.000000
mean,9.506684,5.514706,1378.676471,34.282086,0.237968
std,8.095396,5.839307,1459.826781,24.376714,0.426124
min,0.000000,1.000000,250.000000,2.000000,0.000000
25%,2.750000,2.000000,500.000000,16.000000,0.000000
50%,7.000000,4.000000,1000.000000,28.000000,0.000000
75%,14.000000,7.000000,1750.000000,50.000000,0.000000
max,74.000000,50.000000,12500.000000,98.000000,1.000000


In [16]:
df.isna().sum()
print(df.dtypes)

Recency          int64
Frequency        int64
Monetary         int64
Time             int64
Donated_Blood    int64
dtype: object


In [18]:
df['Donated_Blood'].value_counts()

,count
Donated_Blood,
0,570
1,178


In [7]:
column_names = ['Recency', 'Frequency', 'Monetary', 'Time', 'Donated_Blood']
df = pd.read_csv(data_url, names=column_names)
df['Time'] = pd.to_numeric(df['Time'], errors='coerce')
df = df.dropna(subset=['Time'])
grouped = df.groupby('Monetary', as_index=False)['Time'].mean()
grouped_sorted = grouped.sort_values(by='Monetary', ascending=False)
print(grouped_sorted)

   Monetary       Time
32     9500  98.000000
31     8500  77.000000
30     8250  98.000000
29      750  34.540230
28     6500  76.000000
27     6000  73.333333
26     5750  58.000000
25     5500  63.000000
24     5250  47.000000
23     5000  57.000000
22      500  21.196429
21     4750  65.500000
20     4500  78.000000
19     4250  76.500000
18     4000  72.307692
17     3750  67.500000
16     3500  67.000000
15     3250  49.777778
14     3000  67.071429
13     2750  51.727273
12     2500  46.357143
11      250  12.107595
10     2250  47.416667
9      2000  45.709677
8      1750  51.162791
7      1500  37.423077
6     12500  98.000000
5      1250  36.145161
4     11500  98.000000
3     11000  98.000000
2     10750  86.000000
1     10250  98.000000
0      1000  33.306452


In [9]:
df.groupby('Monetary')['Time'].agg(['mean', 'min', 'max', 'median'])

,mean,min,max,median
Monetary,,,,
1000,33.306452,4.0,87.0,28.0
10250,98.000000,98.0,98.0,98.0
10750,86.000000,86.0,86.0,86.0
11000,98.000000,98.0,98.0,98.0
11500,98.000000,98.0,98.0,98.0
1250,36.145161,11.0,74.0,34.0
12500,98.000000,98.0,98.0,98.0
1500,37.423077,14.0,86.0,35.0
1750,51.162791,14.0,93.0,52.0


In [11]:
df['Monetary'] = pd.to_numeric(df['Monetary'], errors='coerce')
df = df.dropna(subset=['Monetary'])
grouped = df.groupby(['Recency', 'Frequency'], as_index=False)['Monetary'].mean()
print(grouped)

    Recency Frequency  Monetary
0         0        13    3250.0
1         0         2     500.0
2         0        26    6500.0
3         0         3     750.0
4         0         8    2000.0
..      ...       ...       ...
183       9         3     750.0
184       9         4    1000.0
185       9         5    1250.0
186       9         8    2000.0
187       9         9    2250.0

[188 rows x 3 columns]


In [14]:
print('Среднее:', round(df['Monetary'].mean(), 2),
      'Медиана:', df['Monetary'].median(),
      'Мода:', df['Monetary'].mode()[0])

Среднее: 1378.68 Медиана: 1000.0 Мода: 250


In [16]:
df[df.Time > 95.00]

,Recency,Frequency,Monetary,Time,Donated_Blood
1,2,50,12500,98.0,1
10,5,46,11500,98.0,1
138,2,12,3000,98.0,0
242,11,22,5500,98.0,0
328,14,16,4000,98.0,0
342,23,38,9500,98.0,0
504,2,44,11000,98.0,0
506,2,41,10250,98.0,1
518,4,33,8250,98.0,1
563,4,16,4000,98.0,1


In [19]:
df_info = pd.read_csv(data_url, names=column_names)
df = pd.read_csv(data_url, names=column_names)
for d in [df_info, df]:
    d['Monetary'] = pd.to_numeric(d['Monetary'], errors='coerce')
    d.dropna(subset=['Monetary'], inplace=True)
joined_dfs = df_info.set_index('Monetary').join(
    df.set_index('Monetary'),
    how='inner',
    lsuffix='_left',
    rsuffix='_right'
).reset_index()
print(joined_dfs.head(5))

   Monetary Recency_left Frequency_left Time_left Donated_Blood_left  \
0   12500.0            2             50        98                  1   
1    3250.0            0             13        28                  1   
2    3250.0            0             13        28                  1   
3    3250.0            0             13        28                  1   
4    3250.0            0             13        28                  1   

  Recency_right Frequency_right Time_right Donated_Blood_right  
0             2              50         98                   1  
1             0              13         28                   1  
2             1              13         47                   0  
3             2              13         53                   1  
4             2              13         76                   1  
